In [3]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
from utils.coords import feet2m
from utils.divtree_reader import readDivideTree

%matplotlib inline

del mpl.font_manager.weight_dict['roman']
mpl.font_manager._rebuild()

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 20
plt.rcParams["font.weight"] = 'normal'
mpl.rcParams['image.cmap'] = 'gray'

KeyError: 'roman'

In [ ]:
filesPath = 'data/dems'

imgOff = 50
edgeColor = (0.75,0,0,1)

for f in os.listdir(filesPath):
    
    if not f.endswith('.txt'):
        continue
    
    fpath = os.path.join(filesPath, f)
        
    # read divide tree
    peakCoords, peakElevs, saddleCoords, saddleElevs, saddlePeaks, _ = readDivideTree(fpath, returnDEMCoords=True)
    numPeaks = peakElevs.size
    numSaddles = saddlePeaks.shape[0]
    
    # read image (16 bit PNG, heights in feet)
    img = cv2.imread(fpath.replace('.txt', '.png'), -cv2.IMREAD_ANYDEPTH)
    
    print(f, '%.2f'%feet2m(img.max()), '%.2f'%feet2m(img.min()))
    
    # normalize image
    img = (img - img.min())/(img.max() - img.min())
    
    fig = plt.figure(figsize=(8, 8*img.shape[0]/img.shape[1]))
    ax = fig.add_subplot(111)    

    # plot image
    ax.imshow(img[...], interpolation='bicubic', zorder=0, cmap='gray')

    # plot peaks
    ax.scatter(peakCoords[:,0], img.shape[0] - peakCoords[:,1], marker='^', zorder=3,
               s=20*peakElevs/peakElevs.max(), c='white', edgecolors=(1,0.75,0,1), linewidths=1)
    
    # plot saddles
    ax.scatter(saddleCoords[:,0], img.shape[0] - saddleCoords[:,1], marker='o', 
               c='white', edgecolors=(146/255, 208/255, 80/255, 1), s=6, zorder=2)

    # plot ridges
    for i in range(saddleCoords.shape[0]):
        if saddlePeaks[i,0] < 0 or saddlePeaks[i,1] < 0:
            continue
            
        p1 = peakCoords[saddlePeaks[i,0]]
        p2 = peakCoords[saddlePeaks[i,1]]
        ps = saddleCoords[i]
        
        ax.plot([p1[0], ps[0]], [img.shape[0] - p1[1], img.shape[0] - ps[1]], color=edgeColor, linewidth=1, zorder=1)
        ax.plot([p2[0], ps[0]], [img.shape[0] - p2[1], img.shape[0] - ps[1]], color=edgeColor, linewidth=1, zorder=1)

    # place a text box in upper left in axes coords
    #ax.text(0.016, 0.016, f.split('.')[0].replace('-', ' '), transform=ax.transAxes, fontsize=24,
    #        verticalalignment='bottom', horizontalalignment='left',
    #        bbox=dict(boxstyle='square', facecolor='white', alpha=1))

    plt.xlim(peakCoords[:,0].min() + imgOff, peakCoords[:,0].max() - imgOff)
    plt.ylim(peakCoords[:,1].max() - imgOff, peakCoords[:,1].min() + imgOff)

    ax.set_xticks([])
    ax.set_yticks([])
    
    fig.savefig('divtree_%s.pdf' % f.split('.')[0], dpi=300, bbox_inches='tight', pad_inches=0)

alaska.txt 6179.82 280.11
alps-dolomites.txt 3332.68 0.00
alps-ecrins.txt 4080.66 587.35
alps-montblanc.txt 4797.86 455.07
andes-aconcagua.txt 6931.76 1347.83
andes-chile.txt 5948.78 1658.72
appalachians.txt 1661.77 298.40
colorado.txt 4384.85 2047.04
gobi.txt 1051.26 847.65
highlands.txt 1308.51 71.32
himalaya-annapurna.txt 8069.28 1026.57
himalaya-everest.txt 8837.37 2362.20
karakoram.txt 8577.99 3352.19
norway.txt 1891.28 0.00
patagonia.txt 2788.92 0.00
pyrenees.txt 3364.38 659.28
rockies.txt 3573.17 799.80
sahara.txt 474.88 189.89
